In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine,text 
import leafmap

# Download data from CSDI Portal

In [ ]:
from fgdbDL import download_and_extract_gdb 

url = "https://static.csdi.gov.hk/csdi-webpage/download/51d63757e2675874af80eef94afb6a35/fgdb"

#storage_path = "/home/steeb/Documents/GIS/"
storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"

download_and_extract_gdb(url, storage_path)

In [ ]:
from shpDL import download_and_extract_shp

url = 'https://static.csdi.gov.hk/csdi-webpage/download/0e55c533715b5da3ae0ca6e6024e90b4/shp'

storage_path = "/home/steeb/Documents/GIS/"
#storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"

download_and_extract_shp(url, storage_path)

# Set up paths and layers + db connection parameters

In [2]:
# Define the paths and layer name (comment out either one gdb_path when not in use)
#gdb_path = "/home/steeb/Documents/GIS/20240509/Building_Footprint.gdb"
gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\Building_Footprint.gdb"

#shp_file = "/home/steeb/Documents/GIS/BDBIAR.shp"
shp_file = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\BDBIAR.shp"

# Define the db connection parameters
username = "postgres"
password = "12345"
host = "localhost"
dbname = "Buildings"
port = "5432"

pg_connection = f"PG:host={host} port={port} dbname={dbname} user={username} password={password}"
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{dbname}")

# Import into a PostgreSQL database using ogr2ogr

In [ ]:
# Imports a GDB into a PostgreSQL database using ogr2ogr
from gdb2pgsql import transfer_gdb_to_postgis

transfer_gdb_to_postgis(gdb_path, pg_connection)

In [ ]:
# Imports a SHP into a PostgreSQL database using ogr2ogr
from shp2pgsql import import_shapefile_to_postgresql

import_shapefile_to_postgresql(shp_file, pg_connection)

In [ ]:
from gdbList import list_layers_with_types

list_layers_with_types(gdb_path)

# Read data from PostgreSQL database into dataframes

In [3]:
table_op = "OCCUPATION_PERMIT"
table_op_blgstr = "OP_BUILDING_STRUCTURE"
table_blgstr = "BUILDING_STRUCTURE"
table_blgcat = "CT_BUILDING_CATEGORY"
table_bdbiar = "BDBIAR"

sql_op = text(f"SELECT * FROM {table_op}")
sql_op_blgstr = text(f"SELECT * FROM {table_op_blgstr}")
sql_blstr = text(f"SELECT * FROM {table_blgstr}")
sql_blgcat = text(f"SELECT * FROM {table_blgcat}")
sql_bdbiar = text(f"SELECT * FROM {table_bdbiar}")

In [4]:
# Read the tables into DataFrames
df_op = pd.read_sql(sql_op, con=engine.connect())
df_op_blgstr = pd.read_sql(sql_op_blgstr, con=engine.connect())
df_blgcat = pd.read_sql(sql_blgcat, con=engine.connect())

# Read the tables with geometry into DataFrames
gdf_blgstr = gpd.read_postgis(sql_blstr, con=engine.connect(), geom_col="shape") 
gdf_bdbiar = gpd.read_postgis(sql_bdbiar, con=engine.connect(), geom_col="wkb_geometry") 

In [5]:
# Select only the "opno" and "opdate" columns from df_op
df_op_subset = df_op[["opno", "opdate"]]

df_op_subset.opdate = pd.to_datetime(df_op_subset["opdate"], utc=True)

# Merge df_op_blgstr with the subset of df_op on the "opno" column
df_merge_op_blgstr = pd.merge(df_op_blgstr,
                            df_op_subset,
                            on="opno",
                            how="right")

In [9]:
# Select only the "buildingstructureid" and "opdate" columns from df_merge_op_blgstr
df_merge_op_blgstr_subset = df_merge_op_blgstr[["buildingstructureid", "opno", "opdate"]]

# Merge gdf_blgstr with the subset of df_merge_op_blgstr on the "buildingstructureid" column
gdf_merge_blgstr = pd.merge(gdf_blgstr,
                df_merge_op_blgstr_subset,
                on="buildingstructureid", how="left")

In [10]:
# Select only the "buildingstructureid" and "opdate" columns from df_merge_op_blgstr
df_blgcat_subset = df_blgcat[["code",
                              "description",
                              "note"]]

df_blgcat_subset = df_blgcat_subset.rename(columns={"code": "category",
                                 "description": "catdesc",
                                 "note": "catnote"})

gdf_merge_blgstr.category = gdf_merge_blgstr.category.astype("object").astype("int64")

# Merge gdf_blgstr with the subset of df_merge_op_blgstr on the "buildingstructureid" column
gdf_merge_blgstr = pd.merge(gdf_merge_blgstr,
                df_blgcat_subset,
                on="category", how="left")

In [11]:
today = pd.to_datetime('today', utc=True).normalize()

gdf_merge_blgstr["calcdate"] = today

In [17]:
gdf_merge_blgstr['age'] = (gdf_merge_blgstr["calcdate"] - gdf_merge_blgstr["opdate"]) / pd.Timedelta(days=365)

In [19]:
# Keep only relevant columns
gdf_merge_blgstr = gdf_merge_blgstr.loc[:, ("buildingstructureid",
                    "buildingcsuid",
                    "buildingstructuretype",
                    "catdesc",
                    "catnote",
                    "status",
                    "officialbuildingnameen",
                    "officialbuildingnametc",
                    "numabovegroundstoreys",
                    "numbasementstoreys",
                    "topheight",
                    "baseheight",
                    "opno",
                    "opdate",
                    "age",
                    "shape")]

# Filter Building structure and Building age by "Tower" type

In [27]:
gdf_blgstr_tower = gdf_merge_blgstr[gdf_merge_blgstr.buildingstructuretype == "T"]
gdf_bdbiar_tower = gdf_bdbiar[gdf_bdbiar.nsearch4_e == "Tower"]

In [28]:
gdf_bdbiar_tower.to_crs(epsg=2326, inplace=True)

In [29]:
gdf_sjoin_blgstr = gpd.sjoin(gdf_blgstr_tower, gdf_bdbiar_tower, how="left")

In [30]:
gdf_sjoin_blgstr.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 333157 entries, 0 to 340950
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   buildingstructureid     333157 non-null  int64              
 1   buildingcsuid           333157 non-null  object             
 2   buildingstructuretype   333157 non-null  object             
 3   catdesc                 333157 non-null  object             
 4   catnote                 333157 non-null  object             
 5   status                  333157 non-null  object             
 6   officialbuildingnameen  59893 non-null   object             
 7   officialbuildingnametc  59348 non-null   object             
 8   numabovegroundstoreys   41884 non-null   float64            
 9   numbasementstoreys      3934 non-null    float64            
 10  topheight               177236 non-null  float64            
 11  baseheight             

In [31]:
# Keep only relevant columns
gdf_sjoin_blgstr = gdf_sjoin_blgstr.loc[:, ("buildingstructureid",
                    "buildingcsuid",
                    "buildingstructuretype",
                    "catdesc",
                    "catnote",
                    "status",
                    "officialbuildingnameen",
                    "officialbuildingnametc",
                    "numabovegroundstoreys",
                    "numbasementstoreys",
                    "topheight",
                    "baseheight",
                    "opno",
                    "opdate",
                    "age",
                    "address_e",
                    "address_c",
                    "search1_e",
                    "search1_c",
                    "search2_e",
                    "search2_c",
                    "nsearch2_e",
                    "nsearch2_c",
                    "nsearch3_e",
                    "nsearch3_c",
                    "nsearch4_e",
                    "nsearch4_c",
                    "nsearch5_e",
                    "nsearch5_c",
                    "shape"
                    )]

In [32]:
gdf_sjoin_blgstr.to_postgis("gdf_sjoin_blgstr", engine, if_exists="replace")

In [37]:
gdf_sjoin_blgstr.nsearch5_e.value_counts()

nsearch5_e
Residential/Composite    34694
Others                    3149
Office/Commercial         2516
Industrial                1735
Name: count, dtype: int64